In [1]:
import random, re
from collections import defaultdict
import itertools
import pycosat
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, BasicAer, IBMQ, execute
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit.circuit.library.standard_gates import ZGate

In [3]:
def tcnfgen(n,k,horn=1):
    cnf = []
    def unique(l,k):
        t = random.randint(1,n)
        while(t in l):
            t = random.randint(1,n)
        return t
    r = (lambda : random.randint(0,1))
    def r_to_sign(x):
        if r() == 1:
            return x
        else:
            return -x
    for i in range(k):
        x = unique([],n)
        y = unique([x],n)
        z = unique([x, y],n)
        if horn:
            cnf.append([x, -y,-z])
        else:
            cnf.append([r_to_sign(x), r_to_sign(y),r_to_sign(z)])
    return cnf

In [4]:
def create_CNF(n, k):
    new_line = []
    line = tcnfgen(n, k)
    for clause in line:
        new_line.append(sorted(clause, key = abs))
    return new_line

In [5]:
def prepare_clause(circuit, clause):
    for element in clause:
        if element<0:
            circuit.x(abs(element))
    return circuit

In [6]:
def or_operator(circuit, clause, counter):
    order_clause = [abs(x) for x in clause]
    for element in order_clause:
        circuit.x(element)
    circuit.mcx(order_clause, counter)
    circuit.x(counter)
    for element in order_clause:
        circuit.x(element)
    return circuit

In [7]:
def oracle(circuit, n, k, CNF):
    counter = n+1 
    for clause in CNF:
        prepare_clause(circuit, clause)
        or_operator(circuit, clause, counter)
        counter = counter + 1
        prepare_clause(circuit, clause)
    return circuit

In [8]:
CCCZ = ZGate().control(2)

In [9]:
def inversion_z(circuit,n):
    for i in range(1,n+1):
        circuit.h(i)

    for i in range(1,n+1):
        circuit.x(i)
    
    circuit.append(CCCZ, [1,2,3])

    for i in range(1,n+1):
        circuit.x(i)

    for i in range(1,n+1):
        circuit.h(i)
    return circuit

In [10]:
n = 3
k = 7
new_line = [[-1,-2,-3],[-1,-2,3],[-1,2,-3],[-1,2,3],[1,-2,-3],[1,-2,3],[1,2,-3]]

In [11]:
for sol in pycosat.itersolve(new_line):
    print(sol)

[-1, -2, -3]


In [17]:
grover_circuit = QuantumCircuit(n+k+1, n+k+1)

for i in range(1, n+1):
    grover_circuit.h(i)

iterations = 3
    
for i in range(iterations):
    oracle(grover_circuit,n, k, new_line)
    grover_circuit.mcx(list(range(n+1, n+k+1)), 0)
    grover_circuit.z(0)
    grover_circuit.inverse()
    grover_circuit.barrier()
    inversion_z(grover_circuit, n)
    grover_circuit.barrier()

    
grover_circuit.measure([1,2,3],[1,2,3])

job = execute(grover_circuit, Aer.get_backend('qasm_simulator'), shots=10000)
#job = execute(mycircuit, backend= simulator_backend, shots=8192)
counts = job.result().get_counts(grover_circuit)
# print the reverse of the outcome
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome, "is observed", counts[outcome], "times")
print("\n")

00110000000 is observed 977 times
00000000000 is observed 3062 times
00010000000 is observed 1008 times
01000000000 is observed 1001 times
01110000000 is observed 1022 times
00100000000 is observed 969 times
01100000000 is observed 1006 times
01010000000 is observed 955 times




In [13]:
grover_circuit.draw()

»
 q_0: ──────────────────────────────────────────────────────────────────────»
      ┌───┐┌───┐┌───┐     ┌───┐┌───┐┌───┐┌───┐     ┌───┐┌───┐┌───┐┌───┐     »
 q_1: ┤ H ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├┤ X ├──■──»
      ├───┤├───┤├───┤  │  ├───┤├───┤├───┤├───┤  │  ├───┤├───┤├───┤└───┘  │  »
 q_2: ┤ H ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├───────■──»
      ├───┤├───┤├───┤  │  ├───┤├───┤├───┤└───┘  │  ├───┤├───┤├───┤       │  »
 q_3: ┤ H ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├───────■──┤ X ├┤ X ├┤ X ├───────■──»
      └───┘└───┘└───┘┌─┴─┐├───┤└───┘└───┘       │  └───┘└───┘└───┘       │  »
 q_4: ───────────────┤ X ├┤ X ├─────────────────┼────────────────────────┼──»
                     └───┘└───┘               ┌─┴─┐┌───┐                 │  »
 q_5: ────────────────────────────────────────┤ X ├┤ X ├─────────────────┼──»
                                              └───┘└───┘               ┌─┴─┐»
 q_6: ─────────────────────────────────────────────────────────────────┤ X ├»
                                                                       └───┘»
 q_7: ──────────────────────────────────────────────────────────────────────»
                                                                            »
 q_8: ──────────────────────────────────────────────────────────────────────»
                                                                            »
 q_9: ──────────────────────────────────────────────────────────────────────»
                                                                            »
q_10: ──────────────────────────────────────────────────────────────────────»
                                                                            »
c: 11/══════════════════════════════════════════════════════════════════════»
                                                                            »
«                                                                            »
« q_0: ──────────────────────────────────────────────────────────────────────»
«      ┌───┐┌───┐┌───┐┌───┐     ┌───┐┌───┐┌───┐     ┌───┐┌───┐               »
« q_1: ┤ X ├┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ X ├────────────■──»
«      ├───┤├───┤└───┘└───┘  │  ├───┤├───┤├───┤  │  ├───┤├───┤┌───┐┌───┐  │  »
« q_2: ┤ X ├┤ X ├────────────■──┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├┤ X ├──■──»
«      ├───┤├───┤┌───┐       │  ├───┤├───┤├───┤  │  ├───┤├───┤├───┤└───┘  │  »
« q_3: ┤ X ├┤ X ├┤ X ├───────■──┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├───────■──»
«      └───┘└───┘└───┘       │  └───┘└───┘└───┘  │  └───┘└───┘└───┘       │  »
« q_4: ──────────────────────┼───────────────────┼────────────────────────┼──»
«                            │                   │                        │  »
« q_5: ──────────────────────┼───────────────────┼────────────────────────┼──»
«      ┌───┐                 │                   │                        │  »
« q_6: ┤ X ├─────────────────┼───────────────────┼────────────────────────┼──»
«      └───┘               ┌─┴─┐┌───┐            │                        │  »
« q_7: ────────────────────┤ X ├┤ X ├────────────┼────────────────────────┼──»
«                          └───┘└───┘          ┌─┴─┐┌───┐                 │  »
« q_8: ────────────────────────────────────────┤ X ├┤ X ├─────────────────┼──»
«                                              └───┘└───┘               ┌─┴─┐»
« q_9: ─────────────────────────────────────────────────────────────────┤ X ├»
«                                                                       └───┘»
«q_10: ──────────────────────────────────────────────────────────────────────»
«                                                                            »
«c: 11/══════════════════════════════════════════════════════════════════════»
«                                                                            »
«                                    ┌───┐┌───┐ ░                         ░ »
« q_0: ──────────────────────────────┤ X ├┤ Z ├─░────────────────────────